In [10]:
import pre_processing
from sklearn.model_selection import train_test_split
import torch
import resnet_encoder
import transformer_decoder

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
processed_data = pre_processing.PreProcess('flickr8k/captions.txt')
print(processed_data.vocab_size)
processed_data.data.head()

8921


,image,caption,cleaned_captions,tokenized_captions
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,"[<start>, a, child, in, a, pink, dress, is, cl...","[2, 1, 44, 5, 1, 92, 172, 8, 121, 55, 1, 398, ..."
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,"[<start>, a, girl, going, into, a, wooden, bui...","[2, 1, 20, 317, 66, 1, 196, 119, 4, 3, 0, 0, 0..."
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,"[<start>, a, little, girl, climbing, into, a, ...","[2, 1, 42, 20, 121, 66, 1, 196, 2438, 4, 3, 0,..."
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...,"[<start>, a, little, girl, climbing, the, stai...","[2, 1, 42, 20, 121, 6, 395, 22, 62, 2438, 4, 3..."
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...,"[<start>, a, little, girl, in, a, pink, dress,...","[2, 1, 42, 20, 5, 1, 92, 172, 317, 66, 1, 196,..."


In [5]:
train_data, val_data = train_test_split(processed_data.data, test_size = 0.1, random_state = 42)
train_data.reset_index(drop = True, inplace = True)
val_data.reset_index(drop = True, inplace = True)
train_data.shape, val_data.shape

((36409, 4), (4046, 4))

In [6]:
unq_data = processed_data.data[['image']].drop_duplicates()
unq_train_data = train_data[['image']].drop_duplicates()
unq_val_data = val_data[['image']].drop_duplicates()
unq_data.shape, unq_train_data.shape, unq_val_data.shape

((8091, 1), (8091, 1), (3323, 1))

In [6]:
resnet_encoder.get_feature(unq_data, 'flickr8k/Images', device, batch_size=1)

  0%|          | 0/8091 [00:00<?, ?it/s]c:\Users\mayan\anaconda3\envs\ai\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mayan\anaconda3\envs\ai\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  8%|▊         | 617/8091 [03:27<41:55,  2.97it/s]  


KeyboardInterrupt: 

In [7]:
num_epochs = 30
batch_size = 32
learning_rate = 0.00001

In [9]:
decoderModel = transformer_decoder.decoderTransformer(16, 4, processed_data.vocab_size, 512, processed_data.max_seq_length, device).to(device)
optimizer = torch.optim.Adam(decoderModel.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.8, patience = 2, verbose = True)
criterion = torch.nn.CrossEntropyLoss(reduction = 'none')
min_val_loss = float('inf')